# Process to Handle and Test Data for YouTube Influencers Dataset in Jupyter Lab

In [1]:
import pandas as pd

# reading csv file 
df = pd.read_csv("youtube_data_united-states.csv",  header=None)
df.head()

,0
0,#;NAME;FOLLOWERS;ER;COUNTRY;TOPIC OF INFLUENCE...
1,1;T-Series @UCq-Fj5jknLsUf-MWSy4_brA;234M;-;In...
2,2;Cocomelon - Nursery Rhymes @UCbCmjCuTUZos6In...
3,3;SET India @UCpEhnqL0y41EpW2TvWAHD7Q;139M;-;I...
4,4;PewDiePie @UC-lHJZR3Gqxm24_Vd_AJ5Yw;111M;0.1...


In [2]:
# File path
file_path = "youtube_data_united-states.csv"

# Load specific columns for analysis
df = pd.read_csv(
    file_path,
    sep=';', 
    usecols=[1, 2, 4, 6],  
    names=['Channel_name', 'Total_subscribers', 'Country', 'Potential_reach'],  
    header=0,
    na_values=['?'] 
)

# Display the first few rows
df.head()


,Channel_name,Total_subscribers,Country,Potential_reach
0,T-Series @UCq-Fj5jknLsUf-MWSy4_brA,234M,India,70.2M
1,Cocomelon - Nursery Rhymes @UCbCmjCuTUZos6Inko...,144M,United States,43.2M
2,SET India @UCpEhnqL0y41EpW2TvWAHD7Q,139M,India,41.7M
3,PewDiePie @UC-lHJZR3Gqxm24_Vd_AJ5Yw,111M,United States,33.3M
4,MrBeast @UCX6OQ3DkcsbYNE6H8uQQuVA,100M,United States,30M


In [3]:
# Check missing values by variable
df.isnull().sum()

Channel_name          0
Total_subscribers     0
Country              25
Potential_reach       0
dtype: int64

In [4]:
# Replace NaN values in the COUNTRY column with 'Unknown'
df['Country'] = df['Country'].fillna('Unknown')

In [5]:
# Extract text before @ in the  the entire column Channel_name 
df['Channel_name'] = df['Channel_name'].str.split(' @').str[0]

# Display the updated DataFrame
df.head()

,Channel_name,Total_subscribers,Country,Potential_reach
0,T-Series,234M,India,70.2M
1,Cocomelon - Nursery Rhymes,144M,United States,43.2M
2,SET India,139M,India,41.7M
3,PewDiePie,111M,United States,33.3M
4,MrBeast,100M,United States,30M


In [6]:
# Convert 'Total_subscribers' and 'Potential_reach' to integers
df['Total_subscribers'] = (
    df['Total_subscribers']
    .str.replace('M', '', regex=False)  # Remove 'M'
    .astype(float)                      # Convert to float
    * 1_000_000                         # Convert to absolute numbers
).astype(int)                           # Convert to integer

df['Potential_reach'] = (
    df['Potential_reach']
    .str.replace('M', '', regex=False)
    .astype(float)
    * 1_000_000
).astype(int)

# Display the updated DataFrame
print(df.head())


                 Channel_name  Total_subscribers        Country  \
0                    T-Series          234000000          India   
1  Cocomelon - Nursery Rhymes          144000000  United States   
2                   SET India          139000000          India   
3                   PewDiePie          111000000  United States   
4                     MrBeast          100000000  United States   

   Potential_reach  
0         70200000  
1         43200000  
2         41700000  
3         33299999  
4         30000000  


In [7]:
# Check data type
df.dtypes

Channel_name         object
Total_subscribers     int64
Country              object
Potential_reach       int64
dtype: object

In [8]:
# Save the updated Dataframe as a new CSV file
output_file_path = "youtube_data_cleaned.csv"
df.to_csv(output_file_path, index=False) 

print(f"DataFrame saved to {output_file_path}")

DataFrame saved to youtube_data_cleaned.csv


# Process to Handle and Test Data for YouTube Influencers Dataset in SQL 

#### 1. Select Relevant Data

USE youtube_usa_db;

SELECT 

    NAME,
    
    COUNTRY2,
    
    Total_subscribers,
    
    Potential_reach
    
FROM

    top_us_youtubers_2024;




#### 2. Data Transformation
-- Extract the position of '@' in NAME

SELECT CHARINDEX('@', NAME), NAME FROM top_us_youtubers_2024;

-- Extract the position of 'M' in Total_subscribers

SELECT CHARINDEX('M', Total_subscribers), Total_subscribers FROM top_us_youtubers_2024;

#### Rename Columns for Clarity
-- Rename 'Potential_reach' to 'total_views'

EXEC sp_rename 'dbo.top_us_youtubers_2024.Potential_reach', 'total_views', 'COLUMN';

-- Rename 'Total_subscribers' to 'total_subscribers'

EXEC sp_rename 'dbo.top_us_youtubers_2024.Total_subscribers', 'total_subscribers', 'COLUMN';

-- Rename 'COUNTRY2' to 'country'

EXEC sp_rename 'dbo.top_us_youtubers_2024.COUNTRY2', 'country', 'COLUMN';

#### Add Numeric Columns
ALTER TABLE dbo.top_us_youtubers_2024

ADD total_views_numeric FLOAT,

    total_subscribers_numeric FLOAT;

#### Update Numeric Columns
UPDATE dbo.top_us_youtubers_2024

SET 
    total_views_numeric = CAST(SUBSTRING(total_views, 1, CHARINDEX('M', total_views) - 1) AS FLOAT),
    
    total_subscribers_numeric = CAST(SUBSTRING(total_subscribers, 1, CHARINDEX('M', total_subscribers) - 1) AS FLOAT);

#### Add Integer Columns
ALTER TABLE dbo.top_us_youtubers_2024

ADD total_views_int INT,

    total_subscribers_int INT;

UPDATE dbo.top_us_youtubers_2024

SET 
    total_views_int = CAST(REPLACE(total_views, 'M', '') AS FLOAT) * 1000000,
    
    total_subscribers_int = CAST(REPLACE(total_subscribers, 'M', '') AS FLOAT) * 1000000;

#### Drop Old Columns
ALTER TABLE dbo.top_us_youtubers_2024

DROP COLUMN total_views, total_subscribers;

#### Rename New Columns
EXEC sp_rename 'dbo.top_us_youtubers_2024.total_views_int', 'total_views', 'COLUMN';

EXEC sp_rename 'dbo.top_us_youtubers_2024.total_subscribers_int', 'total_subscribers', 'COLUMN';


####  3. Create a View

CREATE VIEW view_us_youtubers_2024 AS

SELECT 

    CAST(SUBSTRING(NAME, 1, CHARINDEX('@', NAME)-1) AS varchar(100)) as channel_name,
    
    country,
    
    total_subscribers,
    
    total_views
    
FROM 

    top_us_youtubers_2024;


#### 4. Test the Processed Data

#### Row Count Check
SELECT COUNT(*) as no_of_rows

FROM view_us_youtubers_2024;

#### Column Count Check
SELECT COUNT(*) as column_count 

FROM 

    INFORMATION_SCHEMA.COLUMNS
    
WHERE 

    TABLE_NAME = 'view_us_youtubers_2024';
   
#### Data Type Check

SELECT 

    COLUMN_NAME,
    
    DATA_TYPE
    
FROM 
    INFORMATION_SCHEMA.COLUMNS
    
WHERE 

    TABLE_NAME = 'view_us_youtubers_2024';

#### Duplicate Records Check
SELECT channel_name,

    COUNT(*) as duplicate_count
    
FROM 

    view_us_youtubers_2024
    
GROUP BY 

    channel_name
    
HAVING 

    COUNT(*) > 1;

# Connecting SQL Server Data to Power BI and Calculating Total Subscribers

## **Step 1: Setting up SQL Server for Power BI Connection**

1. **Ensure SQL Server is Running**: Start your SQL Server and verify the server is accessible.
2. **Enable Successful Logins**:
   - Open SQL Server Management Studio (SSMS).
   - Go to the **Server Properties** -> **Security** tab.
   - Set **Login Auditing** to **Successful logins only**.
3. **Configure User Authentication**:
   - Create a new login in **SSMS** under **Security** -> **Logins**.
   - Select **SQL Server Authentication**, set a username and password, and map the user to the database.
   - Grant necessary database permissions (e.g., `db_owner`).
4. **Ensure TCP/IP Protocol is Enabled**:
   - Open **SQL Server Configuration Manager**.
   - Under **SQL Server Network Configuration**, enable the **TCP/IP** protocol.

---

## **Step 2: Connect SQL Server Data to Power BI**

1. Open **Power BI Desktop**.
2. Navigate to **Home** -> **Get Data** -> **SQL Server**.
3. In the **SQL Server Database** dialog:
   - Enter the **Server name** (e.g., `.\\SQLEXPRESS`).
   - Input the **Database name** (e.g., `youtube_usa_db`).
   - Choose **SQL Server Authentication** and provide the username and password.
4. Click **Connect**. Select the tables or views you want to import (e.g., `view_us_youtubers_2024`).
5. Load the data into Power BI for further analysis.

---

## **Step 3: Calculate Total Subscribers and Total Views in Power BI**

### **DAX Formula for Total Subscribers (in Millions)**

1. Open Power BI and go to the **Modeling** tab.
2. Click **New Measure** and enter the following DAX formula:

```DAX
Total Subscriber (M) = 
VAR million = 1000000
VAR sumOfSubscribers = SUM(view_us_youtubers_2024[total_subscribers])
VAR totalSubscribers = DIVIDE(sumOfSubscribers, million)

RETURN totalSubscribers

### **DAX Formula for Total Views (in Millions)**

1. Open Power BI and go to the **Modeling** tab.
2. Click **New Measure** and enter the following DAX formula:

```DAX
Total Views (M) = 
VAR million = 1000000
VAR sumOfTotalViews = SUM(view_us_youtubers_2024[total_views])
VAR totalViews = DIVIDE(sumOfTotalViews, million)

RETURN totalViews

## **Step 4: Calculate Avg Views per Subscriber and Subscriber Engagement Rate in Power BI**

### **DAX Formula for Avg Views per Subscriber**

1. Open Power BI and go to the **Modeling** tab.
2. Click **New Measure** and enter the following DAX formula:

```DAX
Avg Views per Subscriber = 
VAR totalViews = SUM(view_us_youtubers_2024[total_views])
VAR totalSubscribers = SUM(view_us_youtubers_2024[total_subscribers])
VAR finalAvg = DIVIDE(totalViews, totalSubscribers, BLANK())

RETURN finalAvg

### **Subscriber Engagement Rate**

1. Open Power BI and go to the **Modeling** tab.
2. Click **New Measure** and enter the following DAX formula:

```DAX
Subscriber Engagement Rate (%) = 
VAR totalViews = SUM(view_us_youtubers_2024[total_views])
VAR totalSubscribers = SUM(view_us_youtubers_2024[total_subscribers])
VAR engagementRate = DIVIDE(totalViews, totalSubscribers, BLANK()) * 100

RETURN engagementRate

# Power BI Dashboard: Top U.S. YouTubers 2024

## **Overview**
This dashboard provides insights into the top U.S. YouTubers for 2024. It includes detailed metrics on **total views**, **total subscribers**, and **channel engagement ratios**, helping to analyze audience interaction across platforms.

---

## **Key Visualizations**

### **1. Channel Engagement Table**
- **Metrics**:
  - **Channel Name**: The name of the YouTube channel.
  - **Total Views (M)**: Total views of the channel in millions.
  - **Total Subscribers (M)**: Total subscribers of the channel in millions.
  - **Country**: The country of origin for the channel.

### **2. Top 10 YouTubers by Views (Tree Map)**
- Shows the channels with the highest total views.
- Channels are categorized by country, allowing for regional comparisons.

### **3. Top 10 YouTubers by Subscribers (Bar Chart)**
- Highlights the channels with the most subscribers.
- The horizontal bar chart provides a visual comparison of subscriber counts.

### **4. Channel Engagement Ratios (Cards)**
- **Avg Views per Subscriber (M)**:
  - Represents the average number of views per subscriber.
  - Calculated using the DAX formula:
    ```DAX
    Avg Views per Subscriber (M) = 
    VAR totalViews = SUM(view_us_youtubers_2024[total_views])
    VAR totalSubscribers = SUM(view_us_youtubers_2024[total_subscribers])
    VAR finalAvg = DIVIDE(totalViews, totalSubscribers, BLANK())
    RETURN finalAvg
    ```
- **Subscriber Engagement Rate (%)**:
  - Shows the engagement rate of subscribers, calculated as:
    ```DAX
    Subscriber Engagement Rate (%) = 
    VAR totalViews = SUM(view_us_youtubers_2024[total_views])
    VAR totalSubscribers = SUM(view_us_youtubers_2024[total_subscribers])
    VAR engagementRate = DIVIDE(totalViews, totalSubscribers, BLANK()) * 100
    RETURN engagementRate
    ```
- **Total Views (M)**:
  - Displays the sum of all views across channels in millions.

---

## **Data Preparation**

### **Data Source**
- The data is sourced from the **SQL Server** database `youtube_usa_db`.
- Data was imported into Power BI using a direct SQL query on the `view_us_youtubers_2024` view.

### **SQL Query for View**
```sql
CREATE VIEW view_us_youtubers_2024 AS
SELECT 
    CAST(SUBSTRING(NAME, 1, CHARINDEX('@', NAME)-1) AS varchar(100)) AS channel_name,
    country,
    total_subscribers,
    total_views
FROM 
    top_us_youtubers_2024;


# Power BI Dashboard: Top U.S. YouTubers 2024

![Dashboard](dashboard.png)
